In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import glob
import re
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

In [ ]:
# =============================
# CONFIG
# =============================
MONTH_FOLDERS = [
    r"C:/Users/admin1/Desktop/Performance-Tracking/DSP_H2_2025/Tháng 9",  
    
]
SHEET_ID = "1lZ4dsi94HaeWshsEizKTyNHeOOG0tpLJhzL9pMxvd6k"
WORKSHEET_NAME = "Raw_DSP_H2_2025"

CREDENTIALS_PATH = "C:/Users/admin1/Downloads/new_credential.json"  # file JSON key

# =============================
# HELPER FUNCTION
# =============================

def extract_date_from_filename(filename):
    """
    Detect chuỗi yyyyMMdd trong tên file và chuyển sang datetime
    """
    match = re.search(r"\d{8}", filename)
    if match:
        return datetime.strptime(match.group(), "%Y%m%d").date()
    return None

def process_file(file_path):
    """
    Đọc 1 file excel và xử lý theo logic yêu cầu
    """
    df = pd.read_excel(file_path)

    # Bỏ cột Creative Asset nếu có
    if "Creative Asset" in df.columns:
        df = df.drop(columns=["Creative Asset"])

    # Bỏ hàng cuối cùng
    df = df.iloc[:-1, :]

    # Thêm cột ASIN từ cột Creative
    if "Creative" in df.columns:
        df.insert(0, "ASIN", df["Creative"].astype(str).str[:10])
    else:
        raise ValueError(f"'Creative' column not found in {file_path}")

    # Thêm cột Date từ filename
    file_date = extract_date_from_filename(os.path.basename(file_path))
    df.insert(df.columns.get_loc("Creative") + 1, "Date", file_date)

    return df

# =============================
# MAIN LOGIC
# =============================

all_data = []

for folder in MONTH_FOLDERS:
    files = glob.glob(os.path.join(folder, "*.xlsx"))
    for f in files:
        try:
            df = process_file(f)
            all_data.append(df)
        except Exception as e:
            print(f"❌ Lỗi khi xử lý {f}: {e}")

# Merge tất cả
merged_df = pd.concat(all_data, ignore_index=True)

# =============================
# PUSH TO GOOGLE SHEETS
# =============================

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(CREDENTIALS_PATH, scopes=scope)
client = gspread.authorize(creds)

sh = client.open_by_key(SHEET_ID)

try:
    worksheet = sh.worksheet(WORKSHEET_NAME)
    worksheet.clear()
except gspread.exceptions.WorksheetNotFound:
    worksheet = sh.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

set_with_dataframe(worksheet, merged_df)

print("✅ Merge và push dữ liệu thành công lên Google Sheets!")


✅ Merge và push dữ liệu thành công lên Google Sheets!


In [2]:
import pandas as pd
import os
import glob
import re
from datetime import datetime
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

# =============================
# CONFIG
# =============================
MONTH_FOLDER = r"C:/Users/admin1/Desktop/Performance-Tracking/DSP_H2_2025/Tháng 9"   # đổi path cho đúng
SHEET_ID = "1lZ4dsi94HaeWshsEizKTyNHeOOG0tpLJhzL9pMxvd6k"
WORKSHEET_NAME = "Raw_DSP_H2_2025"
CREDENTIALS_PATH = "C:/Users/admin1/Downloads/new_credential.json"  # file JSON key

# =============================
# HELPER FUNCTION
# =============================

def extract_date_from_filename(filename):
    """
    Detect chuỗi yyyyMMdd trong tên file và chuyển sang datetime
    """
    match = re.search(r"\d{8}", filename)
    if match:
        return datetime.strptime(match.group(), "%Y%m%d").date()
    return None

def process_file(file_path):
    """
    Đọc 1 file excel và xử lý theo logic yêu cầu
    """
    df = pd.read_excel(file_path)

    # Bỏ cột Creative Asset nếu có
    if "Creative Asset" in df.columns:
        df = df.drop(columns=["Creative Asset"])

    # Bỏ hàng cuối cùng
    df = df.iloc[:-1, :]

    # Thêm cột ASIN từ cột Creative
    if "Creative" in df.columns:
        df.insert(0, "ASIN", df["Creative"].astype(str).str[:10])
    else:
        raise ValueError(f"'Creative' column not found in {file_path}")

    # Thêm cột Date từ filename
    file_date = extract_date_from_filename(os.path.basename(file_path))
    df.insert(df.columns.get_loc("Creative") + 1, "Date", file_date)

    return df

# =============================
# MAIN LOGIC
# =============================

all_data = []

files = glob.glob(os.path.join(MONTH_FOLDER, "*.xlsx"))
for f in files:
    try:
        df = process_file(f)
        all_data.append(df)
    except Exception as e:
        print(f"❌ Lỗi khi xử lý {f}: {e}")

# Merge tất cả file trong Tháng 9
merged_df = pd.concat(all_data, ignore_index=True)

# =============================
# APPEND TO GOOGLE SHEETS
# =============================

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file(CREDENTIALS_PATH, scopes=scope)
client = gspread.authorize(creds)

sh = client.open_by_key(SHEET_ID)
worksheet = sh.worksheet(WORKSHEET_NAME)

# Lấy vị trí dòng cuối cùng
last_row = len(worksheet.get_all_values())

# Append mà KHÔNG ghi header
set_with_dataframe(worksheet, merged_df, row=last_row + 1, include_column_header=False)

print("✅ Append dữ liệu tháng 9 thành công vào Google Sheets!")


✅ Append dữ liệu tháng 9 thành công vào Google Sheets!
